In [1]:
import warnings

import pandas as pd
import numpy as np
import torch

from MLWrappers import SklearnBlackBox
from PrivacyAttacks import MiaPrivacyAttack, LabelOnlyPrivacyAttack, AloaPrivacyAttack
from MLPrivacyEvaluator import PrivacyEvaluator

warnings.simplefilter("ignore", UserWarning)

In [2]:
MODEL_NAME = 'dt'
DS_NAME = 'adult'
DATA_FOLDER = f'./data/{DS_NAME}'

# We load the data used to train and test the model, as well as the shadow data
train_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_set.csv', skipinitialspace=True)
train_label = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_label.csv', skipinitialspace=True)
test_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_test_set.csv', skipinitialspace=True)
shadow_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_shadow_set.csv', skipinitialspace=True)


In [3]:
# we load the target black box model using our wrapper
target = SklearnBlackBox(f'./models/{DS_NAME}_{MODEL_NAME}.pkl')

In [4]:
# We can define the parameters to be passed to the shadow models (in our case random forests)
shadow_forest_params = {'n_estimators': 100}

In [5]:
# We initialise the attacks, with the desired parameters for each
mia = MiaPrivacyAttack(target,
                       n_shadow_models=5)
label_only = LabelOnlyPrivacyAttack(target,
                                    n_shadow_models=5,
                                    shadow_model_type='rf',
                                    shadow_model_params=shadow_forest_params,
                                    n_noise_samples_fit=1000)
aloa = AloaPrivacyAttack(target,
                         n_shadow_models=5,
                         shadow_model_type='rf',
                         shadow_model_params=shadow_forest_params,
                         n_noise_samples_fit=1000,
                         shadow_test_size=0.3,
                         undersample_attack_dataset=True)
attacks = [
    mia,
    label_only,
    aloa
]

In [6]:
# We initialise the PrivacyEvaluator object
# We pass the target model and the list of attacks we want to use
evaluator = PrivacyEvaluator(target, attacks)

In [7]:
# We use the fit() method to execute the attacks, starting from the shadow data
evaluator.fit(shadow_set, save_folder = f'./results_{DS_NAME}_{MODEL_NAME}')

100%|██████████| 3630/3630 [00:30<00:00, 117.78it/s]


In [8]:
# Then we can obtain the performances using the report() method
results = evaluator.report(train_set, test_set)
print(results)

100%|██████████| 24134/24134 [03:28<00:00, 115.80it/s]


{'mia_attack': {'classification_report': {'IN': {'precision': 0.8073199840869911, 'recall': 0.31532604754752164, 'f1-score': 0.45351609058402864, 'support': 19307}, 'OUT': {'precision': 0.20333875730729825, 'recall': 0.6989848767350321, 'f1-score': 0.3150326797385621, 'support': 4827}, 'accuracy': 0.3920609927902544, 'macro avg': {'precision': 0.5053293706971447, 'recall': 0.5071554621412768, 'f1-score': 0.38427438516129536, 'support': 24134}, 'weighted avg': {'precision': 0.6865187335000358, 'recall': 0.3920609927902544, 'f1-score': 0.42581826079406154, 'support': 24134}}}, 'label_only_attack': {'classification_report': {'IN': {'precision': 0.8012559549588566, 'recall': 0.9582534831926245, 'f1-score': 0.872750430454985, 'support': 19307}, 'OUT': {'precision': 0.22796934865900384, 'recall': 0.04930598715558318, 'f1-score': 0.08107647760177142, 'support': 4827}, 'accuracy': 0.7764564514792409, 'macro avg': {'precision': 0.5146126518089302, 'recall': 0.5037797351741038, 'f1-score': 0.476

In [9]:
results['mia_attack']['classification_report']

{'IN': {'precision': 0.8073199840869911,
  'recall': 0.31532604754752164,
  'f1-score': 0.45351609058402864,
  'support': 19307},
 'OUT': {'precision': 0.20333875730729825,
  'recall': 0.6989848767350321,
  'f1-score': 0.3150326797385621,
  'support': 4827},
 'accuracy': 0.3920609927902544,
 'macro avg': {'precision': 0.5053293706971447,
  'recall': 0.5071554621412768,
  'f1-score': 0.38427438516129536,
  'support': 24134},
 'weighted avg': {'precision': 0.6865187335000358,
  'recall': 0.3920609927902544,
  'f1-score': 0.42581826079406154,
  'support': 24134}}

In [10]:
results['label_only_attack']['classification_report']

{'IN': {'precision': 0.8012559549588566,
  'recall': 0.9582534831926245,
  'f1-score': 0.872750430454985,
  'support': 19307},
 'OUT': {'precision': 0.22796934865900384,
  'recall': 0.04930598715558318,
  'f1-score': 0.08107647760177142,
  'support': 4827},
 'accuracy': 0.7764564514792409,
 'macro avg': {'precision': 0.5146126518089302,
  'recall': 0.5037797351741038,
  'f1-score': 0.4769134540283782,
  'support': 24134},
 'weighted avg': {'precision': 0.6865938828361505,
  'recall': 0.7764564514792409,
  'f1-score': 0.7144090792317124,
  'support': 24134}}

In [11]:
results['aloa_attack']['classification_report']

{'IN': {'precision': 0.7999253297934125,
  'recall': 0.9987569275392345,
  'f1-score': 0.8883514154746275,
  'support': 19307},
 'OUT': {'precision': 0.14285714285714285,
  'recall': 0.0008286720530350114,
  'f1-score': 0.0016477857878475798,
  'support': 4827},
 'accuracy': 0.7991630065467805,
 'macro avg': {'precision': 0.4713912363252777,
  'recall': 0.4997927997961348,
  'f1-score': 0.44499960063123756,
  'support': 24134},
 'weighted avg': {'precision': 0.6685062472401112,
  'recall': 0.7991630065467805,
  'f1-score': 0.7110033413676379,
  'support': 24134}}